# Movie Recommender
## Saeed Hosseini

<saeed144.73@gmail.com>


Movie Recommender is a code that recommends movies to watch according to IMDB users ratings, movies data like genres and year, and your profile.

Thanks to Jadi and Saeed Aghabozorgi from Google.

v1.0.0

9-Nov-2021

<h1 style="text-align: center; color: red;">START</h1>

<h2 style="display: inline;">Step 1)</h2><h2 style="display: inline; text-align: left; color: blue;">READ PURE DATA SET</h2>

In [10]:
import pandas as pd

movies_df = pd.read_csv('movies_dataset.csv')
rating_df = pd.read_csv('ratings_dataset.csv')

In [48]:
movies_df.head(20)

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995
5,6,Heat,"[Action, Crime, Thriller]",1995
6,7,Sabrina,"[Comedy, Romance]",1995
7,8,Tom and Huck,"[Adventure, Children]",1995
8,9,Sudden Death,[Action],1995
9,10,GoldenEye,"[Action, Adventure, Thriller]",1995


In [12]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


<h2 style="display: inline;">Step 2)</h2><h2 style="display: inline; text-align: left; color: blue;">DATA CLEANSING</h2>

**Movie titles have been merged with production year, let's split them**

In [13]:
#Using Regex -> \d means number and \( or \) means parentheses
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

C:\Users\USER1-~1\AppData\Local\Temp/ipykernel_7592/3883756316.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


**We see movies dataframe has a column that has merged genres data, we should split them too**

In [14]:
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


**We like work with numeric items, we should clean genres column**

**We make a column for each genre and give 1 if the movie is on that genre and give 0 otherwise**

In [19]:
detailed_movies_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        detailed_movies_df.at[index, genre] = 1
detailed_movies_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,NaN,1.0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,NaN,NaN,NaN,1.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Father of the Bride Part II,[Comedy],1995,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Oops! Some movies are not in some genres, we see NaN value for these items**

**Now we fill NAN values with 0 for movies that are not in specific genres**

In [21]:
detailed_movies_df = detailed_movies_df.fillna(0)
detailed_movies_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Now let's look at at rating dataframe**

**we do not need timestamp, let's remove it**

In [23]:
rating_df = rating_df.drop('timestamp', 1)
rating_df.head()

C:\Users\USER1-~1\AppData\Local\Temp/ipykernel_7592/1587516875.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  rating_df = rating_df.drop('timestamp', 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


<h2 style="display: inline;">Step 3)</h2><h2 style="display: inline; text-align: left; color: blue;">ML Algorithms</h2>

**We use som sample data to test our algorithm**

In [49]:
sample_user_input = [
            {'title':'Casino', 'rating':4},
            {'title':'Toy Story', 'rating':5},
            {'title':'Jumanji', 'rating':3.5},
            {'title':"Pulp Fiction", 'rating':1},
            {'title':'Akira', 'rating':2}
         ] 
sample_user_df = pd.DataFrame(sample_user_input)
sample_user_df

,title,rating
0,Casino,4.0
1,Toy Story,5.0
2,Jumanji,3.5
3,Pulp Fiction,1.0
4,Akira,2.0


**Now we check sample user input if there are in our main data frame or not, and attach their movie ID if there are**

In [50]:
#Filtering out the movies by title
inputId = movies_df[movies_df['title'].isin(sample_user_df['title'].tolist())]
print(inputId)
#Then merging it so we can get the movieId. It's implicitly merging it by title.
sample_user_df = pd.merge(inputId, sample_user_df)
#Dropping information we won't use from the input dataframe
sample_user_df = sample_user_df.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
sample_user_df

     movieId         title                                             genres  \
0          1     Toy Story  [Adventure, Animation, Children, Comedy, Fantasy]   
1          2       Jumanji                     [Adventure, Children, Fantasy]   
15        16        Casino                                     [Crime, Drama]   
257      296  Pulp Fiction                   [Comedy, Crime, Drama, Thriller]   
973     1274         Akira             [Action, Adventure, Animation, Sci-Fi]   

     year  
0    1995  
1    1995  
15   1995  
257  1994  
973  1988  


C:\Users\USER1-~1\AppData\Local\Temp/ipykernel_7592/2339038319.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  sample_user_df = sample_user_df.drop('genres', 1).drop('year', 1)


,movieId,title,rating
0,1,Toy Story,5.0
1,2,Jumanji,3.5
2,16,Casino,4.0
3,296,Pulp Fiction,1.0
4,1274,Akira,2.0


**Attach genres to our sample data**

In [51]:
sample_datailed_df = detailed_movies_df[detailed_movies_df['movieId'].isin(sample_user_df['movieId'].tolist())]
sample_datailed_df

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,16,Casino,"[Crime, Drama]",1995,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**We work with content, so let's extract our wanted data**

In [52]:
#Resetting the index to avoid future issues
sample_datailed_df = sample_datailed_df.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
sample_genre_df = sample_datailed_df.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
sample_genre_df

C:\Users\USER1-~1\AppData\Local\Temp/ipykernel_7592/2591190755.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  sample_genre_df = sample_datailed_df.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**Now our data are ready :-)**

<h2 style="display: inline;">Step 4)</h2><h2 style="display: inline; text-align: left; color: blue;">ML Algorithms</h2>

**Let's know what genre our user really likes**

In [53]:
#Dot produt to get weights
user_profile = sample_genre_df.transpose().dot(sample_user_df['rating'])
#The user profile
user_profile

Adventure             10.5
Animation              7.0
Children               8.5
Comedy                 6.0
Fantasy                8.5
Romance                0.0
Drama                  5.0
Action                 2.0
Crime                  5.0
Thriller               1.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 2.0
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

**Our user likes Adventure then Children and Fantasy then Comedy and...**

**Now Let's start by extracting the genre table from the original dataframe**

In [54]:
genre_table = detailed_movies_df.set_index(detailed_movies_df['movieId'])
#And drop the unnecessary information
genre_table = genre_table.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genre_table.head()

C:\Users\USER1-~1\AppData\Local\Temp/ipykernel_7592/12992942.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  genre_table = genre_table.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**With the user's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.**

In [55]:
recommendation_df = ((genre_table*user_profile).sum(axis=1))/(user_profile.sum())
recommendation_df = recommendation_df.sort_values(ascending=False)
recommendation_df.head()

movieId
134853    0.819820
2987      0.819820
108932    0.765766
26340     0.765766
32031     0.765766
dtype: float64

In [56]:
movies_df.loc[movies_df['movieId'].isin(recommendation_df.head(20).keys())]

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
2809,3754,"Adventures of Rocky and Bullwinkle, The","[Adventure, Animation, Children, Comedy, Fantasy]",2000
3000,4016,"Emperor's New Groove, The","[Adventure, Animation, Children, Comedy, Fantasy]",2000
3194,4306,Shrek,"[Adventure, Animation, Children, Comedy, Fanta...",2001
3568,4886,"Monsters, Inc.","[Adventure, Animation, Children, Comedy, Fantasy]",2001
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6194,45074,"Wild, The","[Adventure, Animation, Children, Comedy, Fantasy]",2006
